In [227]:
import pandas as pd
import numpy as np
# from gensim.models import Word2Vec
# from gensim.models.doc2vec import TaggedDocument
import json
from sklearn.model_selection import train_test_split
import string as string
import time
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

In [196]:
doc = pd.read_csv('data/keluhan.csv', encoding='ISO-8859-1')
doc['Keluhan'] = doc['Keluhan'].map({'Ya':1}).fillna(0)
doc['Bukan Keluhan/Respon'] = doc['Bukan Keluhan/Respon'].map({'Ya':1}).fillna(0)
doc['Respon'] = doc['Respon'].map({'Ya':'1'}).fillna(0)
X = doc['Tweet']
y = doc['Keluhan']
doc.head

<bound method NDFrame.head of                 ID                 Timestamp  \
0     6.788450e+17  21 Des 2015, 02:50:25 PM   
1     6.788460e+17  21 Des 2015, 02:53:02 PM   
2     6.788470e+17  21 Des 2015, 02:57:55 PM   
3     6.788470e+17  21 Des 2015, 02:58:54 PM   
4     6.788490e+17  21 Des 2015, 03:06:49 PM   
5     6.788510e+17  21 Des 2015, 03:13:53 PM   
6     6.788510e+17  21 Des 2015, 03:13:58 PM   
7     6.788530e+17  21 Des 2015, 03:23:12 PM   
8     6.788570e+17  21 Des 2015, 03:37:41 PM   
9     6.788630e+17  21 Des 2015, 04:03:26 PM   
10    6.788650e+17  21 Des 2015, 04:11:37 PM   
11    6.788650e+17  21 Des 2015, 04:12:01 PM   
12    6.788660e+17  21 Des 2015, 04:13:05 PM   
13    6.788670e+17  21 Des 2015, 04:19:28 PM   
14    6.788700e+17  21 Des 2015, 04:29:08 PM   
15    6.788710e+17  21 Des 2015, 04:32:37 PM   
16    6.788730e+17  21 Des 2015, 04:41:15 PM   
17    6.788770e+17  21 Des 2015, 04:58:33 PM   
18    6.788820e+17  21 Des 2015, 05:17:40 PM   
19    6.78

In [197]:
def removeURL(raw):
    cleanr = re.compile("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+")
    cleantext = re.sub(cleanr, '', raw)
    
    return cleantext

preprocess = []

for i in range(len(X)):
    preprocess.append(removeURL(X[i]))

X = preprocess

X[1]

'@ridwankamil @dbmpkotabdg kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [198]:
def removeTag(raw):
    cleanr = re.compile("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)")
    cleantext = re.sub(cleanr, '', raw)
    
    return cleantext

preprocess = []

for i in range(len(X)):
    preprocess.append(removeTag(X[i]))

X = preprocess

X[1]

'  kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [199]:
def removedPunctutation(text):
    removed = set(string.punctuation)
    
    return ''.join(w for w in text if w not in removed)

preprocess = []

for i in range(len(X)):
    preprocess.append(removedPunctutation(X[i]))

X = preprocess
X[1]

'  kang teman saya tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [200]:
def removeStopWord(text):
    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    return stopword.remove(text)

preprocess = []

for i in range(len(X)):
    preprocess.append(removeStopWord(X[i]))

X = preprocess
X[1]

'  kang teman tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

In [222]:
def stemWord(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    return stemmer.stem(text)

preprocess = []

for i in range(len(X)):
    preprocess.append(removeStopWord(X[i]))

X = preprocess
X[1]

'  kang teman tertimpa pohn dijln sangkuriang dpn polsek coblong tlg ditertibkan phn yg sdh lapuknuhun'

# Ekstraksi Fitur

In [223]:
count_vect = CountVectorizer()
count_vect.fit(X)
x = count_vect.transform(X)

In [228]:
vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(X) 

In [204]:
text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [205]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [216]:
log_regression = LogisticRegression()
log_regression.fit(x_train, y_train)

log_regression_pred = log_regression.predict(x_test)
print('F1 Score Logistic : ', f1_score(log_regression_pred, y_test))
print('Accuracy Score Logistic : ', accuracy_score(log_regression_pred, y_test))
print('Precision Score Logistic : ', precision_score(log_regression_pred, y_test))
print('Recall Score Logistic : ', recall_score(log_regression_pred, y_test))

F1 Score Logistic :  0.8476621417797889
Accuracy Score Logistic :  0.8926673751328374
Precision Score Logistic :  0.8252569750367107
Recall Score Logistic :  0.8713178294573644


In [217]:
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(x_train, y_train)

neigh_pred = neigh.predict(x_test)
print('F1 Score KNN : ', f1_score(neigh_pred, y_test))
print('Accuracy Score KNN : ', accuracy_score(neigh_pred, y_test))
print('Precision Score KNN : ', precision_score(neigh_pred, y_test))
print('Recall Score KNN : ', recall_score(neigh_pred, y_test))

F1 Score KNN :  0.5571725571725572
Accuracy Score KNN :  0.7736450584484591
Precision Score KNN :  0.3935389133627019
Recall Score KNN :  0.9537366548042705


In [219]:
tree = DecisionTreeClassifier()

tree.fit(x_train, y_train)

tree_pred = tree.predict(x_test)
print('F1 Score DTL : ', f1_score(tree_pred, y_test))
print('Accuracy Score DTL : ', accuracy_score(tree_pred, y_test))
print('Precision Score DTL : ', precision_score(tree_pred, y_test))
print('Recall Score DTL : ', recall_score(tree_pred, y_test))

F1 Score DTL :  0.7651792245793708
Accuracy Score DTL :  0.8294367693942615
Precision Score DTL :  0.7679882525697503
Recall Score DTL :  0.7623906705539358
